In [1]:
import ee
import geemap
import numpy as np
import json
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import geemap.colormaps as geecm

In [4]:
# # # Trigger the authentication flow.
# ee.Authenticate()

# # # Initialize the library.
ee.Initialize()

In [5]:
# define folder and file paths
folder_HMA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"HMA2")
regions_fp = os.path.join(folder_HMA, 'Datasets', 'AOI definition', 'Final AOI', "Final Products", 'subregion_boundaries.shp')
rgi_5km_fp = os.path.join(folder_HMA, 'Datasets', 'AOI definition', 'Final AOI', "Final Products", 'AOI_5km.shp')
fp_dc = os.path.join(folder_HMA, 'Datasets', 'AOI definition', 'Final AOI', "Final Products", 'AOI_dc_rgi.shp')

In [6]:
# open rgi
rgi_gdf = gpd.read_file(fp_dc)

# calculate centroids of each rgi, format lists
centroids = rgi_gdf['geometry'].centroid.to_crs("EPSG:4326").values
rgi_ids = rgi_gdf['rgi_id'].values

In [7]:
# load the climate data, select variables and dates
climate_data = (ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')
                    .filterDate('1980-01-01', '2024-01-01')
                    .select(['temperature_2m', 'total_precipitation_sum'])
               )

# make temp and precip into their own image
image_temp = climate_data.select('temperature_2m').toBands()
image_prec = climate_data.select('total_precipitation_sum').toBands()

In [37]:
# visualize summer temp
t_summer = climate_data.select('temperature_2m').filter(ee.Filter.calendarRange(6,8,'month'))
t_summer_modern = t_summer.filter(ee.Filter.calendarRange(2014,2023,'year')).mean().clip(ee.Geometry.BBox(60, 20, 110, 45))
t_summer_historic = t_summer.filter(ee.Filter.calendarRange(1980,1989,'year')).mean().clip(ee.Geometry.BBox(60, 20, 110, 45))          
t_summer_delta = t_summer_modern.subtract(t_summer_historic)

p_summer = climate_data.select('total_precipitation_sum').filter(ee.Filter.calendarRange(6,8,'month'))
p_summer_modern = p_summer.filter(ee.Filter.calendarRange(2014,2023,'year')).mean().clip(ee.Geometry.BBox(60, 20, 110, 45))
p_summer_historic = p_summer.filter(ee.Filter.calendarRange(1980,1989,'year')).mean().clip(ee.Geometry.BBox(60, 20, 110, 45))          
p_summer_delta = p_summer_modern.subtract(p_summer_historic)

# Create a default map
Map = geemap.Map(basemap='HYBRID')
Map.setCenter(85, 32, 4) 

# Define the visualization parameters.
vizParamsT = {'palette':['blue','white','red'],
             'min': -3, 'max': 3}
vizParamsP = {'palette':['blue','white','red'],
             'min': -1, 'max': 1}

# Center the map and display the image.
# Map.setCenter(-122.1899, 37.5010, 8) # San Francisco Bay
Map.addLayer(t_summer_delta, vizParamsT, 'summerT')
Map.addLayer(p_summer_delta, vizParamsP, 'summerP')

# Display the map
Map

Map(center=[32, 85], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [6]:
# iterate through list of glaciers, sample at location
all_Ts = []
all_Ps = []

for i in range(len(rgi_ids)):
    # if i>2: continue
    rgi_id = rgi_ids[i]
    centroid = centroids[i]

    # format centroid to ee.Geometry
    centroid_ee = ee.Geometry.Point([centroid.x, centroid.y])#.buffer(10)
    
    # sample the temp/precip at this location
    sample_T_i = image_temp.sample(region=centroid_ee, geometries=False, numPixels=1)
    sample_P_i = image_prec.sample(region=centroid_ee, geometries=False, numPixels=1)
    
    # add rgi_id as property
    sample_T_i = sample_T_i.map(lambda f: f.set('rgi_id', rgi_id))
    sample_P_i = sample_P_i.map(lambda f: f.set('rgi_id', rgi_id))
    
    # append to list to be combined later
    all_Ts.append(sample_T_i)
    all_Ps.append(sample_P_i)

# flatten to single FC
all_Ts_fc = ee.FeatureCollection(all_Ts).flatten()
all_Ps_fc = ee.FeatureCollection(all_Ps).flatten()

# save as csv
geemap.ee_to_csv(all_Ts_fc, filename='era5land_monthly_T.csv')
geemap.ee_to_csv(all_Ps_fc, filename='era5land_monthly_P.csv')

Generating URL ...
Please wait ...
Data downloaded to C:\Users\lzell\OneDrive - Colostate\Desktop\HMA2\Scripts\era5land_monthly_T.csv
Generating URL ...
Please wait ...
Data downloaded to C:\Users\lzell\OneDrive - Colostate\Desktop\HMA2\Scripts\era5land_monthly_P.csv
